In [1]:
import os

In [2]:
%pwd

'c:\\Users\\prati\\Desktop\\Project\\English-to-Hindi-Translator\\research'

In [3]:
os.chdir("../")

In [15]:
%pwd

'c:\\Users\\prati\\Desktop\\Project\\English-to-Hindi-Translator'

In [10]:
from src.translator.constants import *
from src.translator.utils.common import read_yaml, create_directories

In [11]:
#update in entity
from dataclasses import dataclass
from pathlib import Path
from transformers import AutoTokenizer
import yaml
import pandas as pd
from datasets import load_dataset

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path_test: Path
    tokenizer_name: str
    source: str

In [12]:
#update configuration manager
class ConfigurationManager:
    def __init__(self, config_filepath='config.yaml'):
        with open(config_filepath, 'r') as file:
            self.config = yaml.safe_load(file)


    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name,
            source=config.source
        )

        return data_transformation_config

In [19]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"

In [20]:
#tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [38]:
dataset = load_dataset("cfilt/iitb-english-hindi")

Repo card metadata block was not found. Setting CardData to empty.


[2023-08-10 11:52:58,039: WARNING: repocard: Repo card metadata block was not found. Setting CardData to empty.]


In [62]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [42]:
max_input_length = 128
max_target_length = 128

source_lang = "en"
target_lang = "hi"

def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [45]:
preprocess_function(dataset["train"][:2])

c:\Users\prati\Desktop\Project\English-to-Hindi-Translator\venv\lib\site-packages\transformers\tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[3872, 85, 2501, 132, 15441, 36398, 0], [32643, 28541, 36253, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]], 'labels': [[63, 2025, 18, 16155, 346, 20311, 24, 2279, 679, 0], [26618, 16155, 346, 33383, 0]]}

In [48]:
#tokenized_datasets = dataset.map(preprocess_function, batched=True)

In [17]:
from transformers import AutoTokenizer
from datasets import load_dataset
from src.translator.entity import DataTransformationConfig

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
        self.source = config.source
        self.save_dir = config.root_dir
        

    def preprocess_function(self,examples):
        max_input_length = 128
        max_target_length = 128

        source_lang = "en"
        target_lang = "hi"
        inputs = [ex[source_lang] for ex in examples["translation"]]
        targets = [ex[target_lang] for ex in examples["translation"]]
        model_inputs = self.tokenizer(inputs, max_length=max_input_length, truncation=True)

        # Setup the tokenizer for targets
        with tokenizer.as_target_tokenizer():
            labels = self.tokenizer(targets, max_length=max_target_length, truncation=True)

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    def save_tokenized_datasets(self, tokenized_datasets):
        os.makedirs(self.save_dir, exist_ok=True)
        tokenized_datasets.save_to_disk(self.save_dir)



In [18]:
try:
    #data_files = {"validation": "validation.csv", "test": "test.csv"}
    #raw_dataset = load_dataset("cfilt/iitb-english-hindi",data_files=data_files)
    raw_dataset = load_dataset("cfilt/iitb-english-hindi")
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    
    tokenized_datasets = raw_dataset.map(data_transformation.preprocess_function, batched=True)

    # Save tokenized datasets to the specified directory
    data_transformation.save_tokenized_datasets(tokenized_datasets)
    
except Exception as e:
    raise e

Repo card metadata block was not found. Setting CardData to empty.


[2023-08-10 15:27:12,994: WARNING: repocard: Repo card metadata block was not found. Setting CardData to empty.]


FileNotFoundError: [Errno 2] No such file or directory: 'config.yaml'

In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2023-07-03 11:53:32,872: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-03 11:53:32,876: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-03 11:53:32,878: INFO: common: created directory at: artifacts]
[2023-07-03 11:53:32,879: INFO: common: created directory at: artifacts/data_transformation]


c:\Users\prati\Desktop\Project\Text-Summarizer-End-to-End-Project\venv\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prati\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\prati\Desktop\Project\Text-Summarizer-End